More At : https://github.com/iSiddharth20/DeepLearning-ImageClassification-Toolkit

In [ ]:
'''
Name of Model to Use
    - Un-Comment the one you want to use
        - Comment the Rest
'''

MODEL_NAME = 'CNN_ResNet50'

In [ ]:
'''
Working Directories
'''
import os

# Directory of Original Dataset
DATA_DIR = '../Dataset'

# Directory where Pickle Files are Stored
PICKLE_DIR = '../PickleFiles/'

# Directory where Models are Stored
MODEL_DIR = '../TrainedModels/'

# Full Path where Manually Sorted Dataset will be Stored (Folder will be Created By Code)
SORT_DIR = '../SortedDataset/'
os.makedirs(os.path.dirname(SORT_DIR), exist_ok=True)

# Directory where Final Graphs will be Stored (Folder will be Created By Code)
GRAPH_DIR = '../OutputGraphs/'
os.makedirs(os.path.dirname(GRAPH_DIR), exist_ok=True)

In [ ]:
'''
Importing Necessary Libraries
'''

# Helpers
import pickle 
import shutil
from HelperFunctions import image_processing

# Disable TensorFlow Warnings from Cell Outputs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Data Handling and Visualization
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Image Processing
import cv2

# Model Prediction
import tensorflow as tf
import efficientnet.tfkeras as efn 

In [ ]:
'''
Importing
    - One-Hot-Encoding
    - Trained Model
Defining
    - The 'Predict Function' to Import Image File and Predict Class Label 
'''

# Importing One-Hot-Encoding 
try:
    with open(PICKLE_DIR+'OHE.pkl', 'rb') as f: OHE = pickle.load(f)
    OHE_classes = OHE.categories_[0]
    OHE_classes = np.array(OHE_classes, dtype=object)
    print('OHE Import Successful')
except:
    print('OHE Import Unsuccessful')
    
# Importing Trained Model
try:
    model = tf.keras.models.load_model(MODEL_DIR+MODEL_NAME+'.h5')
    print('Trained Model Import Successful') 
except:
    print('Trained Model Import Unsuccessful') 
    
# Defining the Predict Function
def predict(img_path):
    IMG_HEIGHT = 400
    IMG_WIDTH = 600
    # Importing the Image
    img = image_processing(img_path)
    resized_img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    # Predicting Damage Class
    y = model.predict(resized_img[np.newaxis, :, :, :])
    return OHE_classes[np.argmax(y)]

In [ ]:
%%capture

'''
Creating a DataFrame containing 
    - File Name, Orignal Class Label, Predicted Class Label, Accurate(Yes/No)
    - Set 'move_files'==True : to Move Incorrectly Classified Image Files to 'SORT_DIR' Directory
Exporting DataFrame (File Name, Original Class Label, Predicted Class Label) as Pickle File
'''

move_files = False

def get_file_detials(main_dir):
    df = pd.DataFrame(columns=['File Name', 'Orignal Class Label', 'Predicted Class Label', 'Accurate'])
    for root, dir, files in os.walk(main_dir):
        if root == main_dir:
            continue
        class_orig = os.path.basename(root)
        for filename in files:
            accurate = 'No'
            file_path = os.path.join(root, filename)
            class_pred = predict(file_path)
            if class_pred == class_orig:
                accurate = 'Yes'
            df.loc[len(df.index)] = [filename, class_orig, class_pred, accurate]
            if move_files==True and accurate == 'No':
                dest_fpath = SORT_DIR+class_pred+'/'+filename
                os.makedirs(os.path.dirname(dest_fpath), exist_ok=True)
                shutil.copy(file_path, dest_fpath) 
    return df

df_conf = get_file_detials(DATA_DIR)

In [ ]:
'''
Summary of Obtained Results
'''

sort_incorrect = len(df_conf[df_conf['Accurate']=='No'])
sort_correct = len(df_conf[df_conf['Accurate']=='Yes'])
accuracy_actual = str(round((sort_correct/len(df_conf))*100,3))

print('Number of Images Sorted Correctly   : ',sort_correct)
print('Number of Images Sorted Incorrectly : ',sort_incorrect)
print("Actual Accuracy : ", accuracy_actual)

In [ ]:
'''
Confusion Matrix of Obtained Results
    - Set 'save_graph'==True : to Export Confusion Matrix to 'GRAPH_DIR' Directory
        - Filename will be : ModelName_AccuracyPercentage.png
'''

save_graph = False

# Creating the confusion matrix
confusion_df = pd.crosstab(df_conf['Orignal Class Label'], df_conf['Predicted Class Label'], rownames=['Actual'], colnames=['Predicted'])
# Flipping the confusion matrix diagonally
confusion_df = confusion_df.iloc[::-1]
# Displaying the flipped confusion matrix using seaborn
sns.heatmap(confusion_df, annot=True, cmap='RdBu_r', fmt='g')
if save_graph==True:
    plt.savefig(GRAPH_DIR+'ConfusionMatrix_'+MODEL_NAME+'_'+accuracy_actual[:-2]+'.png', bbox_inches = 'tight')
plt.show()